In [ ]:
# Título del proyecto

'''

Shark adventures: ¿Quieres sentir la adrenalina de nadar con tiburones?

'''

# Objetivo del proyecto

'''
La empresa Extreme quiere lanzar una nueva actividad: experiencias de riesgo con tiburones y pretende:
-Definir el target: edad y sexo.
-Limitar la región para el lanzamiento del producto.
-Localizar un parámetro que conlleve el ataque de tiburones.
'''

In [ ]:
# Realizamos la importaciones que necesitaremos a lo largo del proyecto

import numpy as np
import pandas as pd

In [ ]:
# Importamos el archivo csv

dataoriginal = pd.read_csv('GSAF5.csv', encoding = "utf-8")

In [ ]:
# Guía de trabajo del proyecto
'''
El presente archivo implementa las siguientes técnicas:

* Eliminación de **registros duplicados** 

* **Examen** de los datos y **comprensión** de sus campos.

* **Selección** de la información relevante y eliminación de la no pertinente mediante la reducción 
de columnas presentes de en el data frame.

* **Cambio de nombre de columnas** por nombres más informativo y que conlleven menos problemas 
a la hora de trabajar la información.

* **Cambio de nombre de columnas** en función de los objetivos del proyecto.

* Elección y creación de **subset objeto de estudio**.

* Análisis del peso de los **valores nulos** en nuestros datos y actuaciones 
en función de las conclusiones: eliminación de columnas y sustitución de los mismos.

* Corrección de **valores incorrectos**.

* Prevención de presencia de columnas con **low variance**

* Modificaciones de **strings**.

* Corrección de **data types**

* Realización de **Custom-sized bins**

* Obtención de conclusiones tras el estudio a través de funciones.

* Combinación de subsets: **merge** y **melt**

* Exportación de los datos a **.csv**.


'''



In [ ]:
# Comenzamos con una perspectiva visual del dataset.

dataoriginal.head()

In [ ]:
# Encontramos y eliminamos registros duplicados

before = len(dataoriginal)
data = dataoriginal.drop_duplicates()
after = len(dataoriginal)
print('Number of duplicate records dropped: ', str(before - after))

# En primera instancia data no presenta registros duplicados.

In [ ]:
# Selección de columnas

''' Reducimos data seleccionando solo las columnas interesantes en un nuevo DataFrame: data.
Descartamos: Name, pdf, href formula, href, case number 1, Case numer 2, Unnamed 23, Unnamed 22.
Las columnas descartadas no presentan información pertinente para los objetivos del proyecto.
'''
col_seleccion = ['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
                 'Activity', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ']
data = dataoriginal[col_seleccion]


In [ ]:
# Cambio de nombre de columnas.

'''
Modificacción por títulos más informativos y menos conflictivos 
a la hora de trabajar con la información.
'''

data.columns


In [ ]:
'''
Cambiamos los nombres de las siguientes columnas:

'Case number':'Case',
'Sex ':'Sex'
'Fatal (Y/N)': 'Fatal'
'Species ': 'Sepecies'

'''


data = data.rename(columns={'Case number':'Case',
                            'Sex ':'Sex',
                            'Fatal (Y/N)': 'Fatal',
                            'Species ' :'Species'})

data.columns


In [ ]:
# Cambiando el orden de las columnas

'''
Cambiamos el orden de las columnas en función del objetivo del proyecto: localizar
las zonas geográficas con los tiburones más peligrosos para vender baños de alto riesgo a los 
amantes de la adrenalina.
'''

column_order = ['Case Number', 'Fatal', 'Injury', 'Type', 'Activity', 'Sex', 'Age',  'Date', 'Year',  
                'Country', 'Area', 'Location', 'Time', 'Species']

data = data[column_order]
data.columns

In [ ]:
data['Country'].value_counts()

In [ ]:
# Elección y creación de subset objeto de estudio

# Comprobamos qué valores pueden contener las columnas 'Year' y 'Cuntry'

data['Year'].unique() 
data['Country'].unique() 

# Comprobamos en qué años y países se dan más ataque

data['Country'].value_counts()
data['Year'].value_counts()

# Realizamos el filtrado

'''Seleccionamos los casos de los últimos 30 años, para tomar decisiones a partir de información actualizada, 
y los países con mayor concentración de casos, es decir, más de 360'''

filtered = data[(data['Year']>= 1998) & 
                ((data['Country'] == 'USA') | (data['Country'] == 'AUSTRALIA') | (data['Country'] == 'SOUTH AFRICA'))]

filtered.head()


In [ ]:
filtered.Type.value_counts()

In [ ]:
''' Nos interesa conocer las causas del ataques: humanas o naturales.
Para ello creamos una columna 'Cause' categorías significativas: natural, artificial o unknown'''

filtered.loc[filtered['Type'].str.startswith('U'), 'Cause'] = 'Natural'
filtered.loc[filtered['Type'].str.startswith('P'), 'Cause'] = 'Artificial'
filtered.loc[filtered['Type'].str.startswith('I'), 'Cause'] = 'Unknown'
filtered.loc[filtered['Type'].str.startswith('B'), 'Cause'] = 'Unknown'
filtered.loc[filtered['Type'].str.startswith('S'), 'Cause'] = 'Unknown'
filtered.head()

In [ ]:
# Analizammos el peso de los valores nulos en nuestros datos

null_cols = filtered.isnull().sum()
null_cols[null_cols > 0]


In [ ]:
# Por la preponderancia de su valores nulos y la escasa aportación al estudio eliminamos la columna 'Species'

drop_cols = list(null_cols[null_cols > 400].index)
filtered = filtered.drop(drop_cols, axis=1)

In [ ]:
# 'Fatal' e 'Injury' tienen un valor nulo, veamos qué ha ocurrido.

null_displ = filtered[(filtered['Fatal'].isnull()==True)]
null_displ = null_displ[['Injury', 'Activity', 'Sex', 'Age', 'Area','Location','Time']]
null_displ['Injury']

# Observamos que dicho valor de fatal debería ser 'N', lo cambiamos.

filtered[(filtered['Fatal'].isnull()==True)] = 'N'



In [ ]:
null_displ = filtered[(filtered['Injury'].isnull()==True)]
null_displ = null_displ[['Injury', 'Fatal', 'Activity', 'Sex', 'Age', 'Area','Location','Time']]
null_displ

# Por sus registros dicho record debe ser eliminado
filtered = filtered.drop(filtered[(filtered['Injury'].isnull()==True)].index)

In [ ]:
''' El resto de valores nulos los sustituimos por ceros, 
excepto en Age y Time que sustituimos por -1 para evitar errores de interpretación.'''

filtered[['Activity', 'Sex', 'Area', 'Location']].fillna(0)
filtered[['Age', 'Time']].fillna(-1)

In [ ]:
filtered.Fatal.value_counts()

In [ ]:
# Evitamos valores incorrectos, estableciendo que en accidentes fatales, Injury marque High

filtered.loc[(filtered['Fatal']== 'Y') & (filtered['Injury']!= 'No injury'), 'Injury'] = 'High'

In [ ]:
# Comprobamos que no hay columnas con low variance

low_variance = []

for col in filtered._get_numeric_data():
    minimum = min(filtered[col])
    ninety_perc = np.percentile(filtered[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)
low_variance

In [ ]:
#Moficación de strings para poder realiza astype()
'''Primero preparamos los datos para poder usar .astype()'''

filtered['Age'] = filtered['Age'].fillna('-1')
filtered['Age'] = filtered['Age'].str.replace('\xa0 ', '-1')
filtered['Age'] = filtered['Age'].str.replace('Teens', '14')
filtered['Age'] = filtered['Age'].str.replace('36 & 26', '30')
filtered['Age'] = filtered['Age'].str.replace('50s', '55')
filtered['Age'] = filtered['Age'].str.replace('40s', '45')
filtered['Age'] = filtered['Age'].str.replace('50s', '55')
filtered['Age'] = filtered['Age'].str.replace('30s', '35')
filtered['Age'] = filtered['Age'].str.replace('60s', '65')
filtered['Age'] = filtered['Age'].str.replace('mid-30s', '35')
filtered['Age'] = filtered['Age'].str.replace('8 or 10', '9')
filtered['Age'] = filtered['Age'].str.replace('6œ', '-1')
filtered['Age'] = filtered['Age'].str.replace('20s', '25')
filtered['Age'] = filtered['Age'].str.replace('12 or 13', '12')
filtered['Age'] = filtered['Age'].str.replace('33 or 37', '35')
filtered['Age'] = filtered['Age'].str.replace('30 or 36', '33')
filtered['Age'] = filtered['Age'].str.replace('teen', '14')
filtered['Age'] = filtered['Age'].str.replace('Teen', '14')
filtered['Age'] = filtered['Age'].str.replace('23 & 20', '21')
filtered['Age'] = filtered['Age'].str.replace('N', '-1')
filtered['Age'] = filtered['Age'].str.replace('mid-35', '35')
print(set(filtered['Age']))

In [ ]:
#Data type correction de Age para poder hacer bins

'''Aplicamos .astype()'''

filtered['Age'] = filtered['Age'].astype('int')
filtered.dtypes


In [ ]:
# Realizamos un Custom-sized bins para 'Age'


mpg_labels = ['Unknown','Young','Adult','Old']
cutoffs = [-10,-1,20,60,90]
bins = pd.cut(filtered['Age'],cutoffs, labels=mpg_labels)
bins.head(10)


In [ ]:
filtered.head()

In [ ]:
# Conclusiones:

filtered['Country'].value_counts()

"""
En los últimos 30 años USA es el país con mayor número de casos:
USA             986
AUSTRALIA       345
SOUTH AFRICA    148
"""

filtered[(filtered['Country']== 'USA')].Area.value_counts()

"""
La experiencia piloto de nuestros baños extremos las realizaremos en Florida y Hawai, pues son las áreas
que mayor número de casos acumula:
Florida              538
Hawaii               122
"""

(filtered['Sex']== 'M').sum()
(filtered['Sex']== 'F').sum()

'''
1174 ataques a hombres frente 240 a mujeres.
Debemos dirigir nuestro producto a hombres.
'''

filtered['Age'].mean()

bins.value_counts().max()

'''La edad media de los atacados 22.39 unido  unido a a que la categoría de edad  con 
mayor concentración de ataque es old, nos hace dirigir el producto al intervalo de 
edades comprendido entre los 20 y los 60 años'''

filtered['Activity'].value_counts()

'''Además la actividad en concreto que debemos fomentar es el surf frent al simple nado.
El surf es la actividad que mayor número de ataques aglutina. A mucha distancia de la segunda, el nado:
Surfing                                        532
Swimming                                       182
'''


In [ ]:
# Combinación de subsets: merge

'''
Contrastamos país y edad media para comprobar nuestra conclusiones
'''

avg_age = filtered.groupby('Country', as_index=False).agg({'Age':'mean'})

avg_age.columns = ['Country', 'Age']

merged = pd.merge(filtered, avg_age, on='Country')



In [ ]:
# Combinación de subset: agrupando por variable

'''
Para visualizar nuestras conclusiones obtenemos un data frame con los ataques producidos en américa

y le realizamos un .merge() con las columnas de interés.

'''

df_usa = filtered[(filtered['Country']== 'USA')]
melted = pd.melt(df_usa, id_vars=['Country','Area','Time'], 
                 value_vars=['Sex','Date','Activity'])
melted.head(10)



In [ ]:
# Exporting DataFrame

# Export comma-separated variable file
export = filtered.to_csv('filtered.csv', index=False)
